# Lab2 - PuLP Library

<b>Information on group members:</b><br>
1) 156071, Martyna Stasiak <br>
2) 156062, Maria Musiał

In [2]:
from pulp import *  
import numpy as np
import pandas as pd

## Introduction - brief tutorial on PuLP

In [3]:
# Create an LpProblem instance; LpMaximize = objective function is to be maximized
model = LpProblem(name="some-problem", sense=LpMaximize)

In [4]:
# Initialize the decision variables. We can set the name and lower bound as well.
# To create an array of variables, you can use comprehensions of LpProblem.dicts.

x1 = LpVariable(name="x1", lowBound=0)
x2 = LpVariable(name="x2", lowBound=0)

In [5]:
# An example expression
expression = 3 * x1 + 2 * x2
type(expression)

pulp.pulp.LpAffineExpression

In [6]:
# An example constraint
constraint = 2 * x1 + 3 * x2 >= 5
type(constraint)

pulp.pulp.LpConstraint

Ok, let's now use PuLP to solve the below problem:
$max$ $4x_1 + 2x_2$ <br>
s.t.<br>
     $1x_1 + 1x_2 \geq 1$ <br>
     $2x_1 + 1x_2 \leq 6$ <br>
     $-1x_1 + x_2 = 1$ <br>
     $x_1 \geq 0$ <br>
     $x_2 \geq 0$ 

In [7]:
# Problem
model = LpProblem(name="test-problem", sense=LpMaximize)

x1 = LpVariable(name="x1", lowBound=0)
x2 = LpVariable(name="x2", lowBound=0)

model += (1 * x1 + 1*x2 >= 1, "#1 constraint")
model += (2 * x1 + 1*x2 <= 6, "#2 constraint")
model += (-1 * x1 + 1*x2 == 1, "#3 constraint")

# Objective function
obj_func = 4*x1 + 2 * x2
model += obj_func

model

test-problem:
MAXIMIZE
4*x1 + 2*x2 + 0
SUBJECT TO
#1_constraint: x1 + x2 >= 1

#2_constraint: 2 x1 + x2 <= 6

#3_constraint: - x1 + x2 = 1

VARIABLES
x1 Continuous
x2 Continuous

In [8]:
# Solve the problem
status = model.solve()

In [9]:
# Print status
print(f"status: {model.status}, {LpStatus[model.status]}")

status: 1, Optimal


In [10]:
# Print objective value
print(f"objective: {model.objective.value()}")

objective: 12.000000199999999


In [11]:
# Print constraint values
for name, constraint in model.constraints.items():  print(f"{name}: {constraint.value()}")

#1_constraint: 3.3333334
#2_constraint: 9.999999983634211e-08
#3_constraint: 0.0


In [12]:
model.variables()

[x1, x2]

In [13]:
print(x1.value())
print(x2.value())

1.6666667
2.6666667


### The same code but using dictionaries and other nice tricks


In [14]:
model = LpProblem(name="some-problem", sense=LpMaximize)

In [15]:
var_names = ["First", "Second"]
x = LpVariable.dicts("x", var_names, 0)
x

{'First': x_First, 'Second': x_Second}

In [16]:
const_names = ["GE", "LE", "EQ"]
sense = [1, -1, 0] # GE, LE, EQ
coefs = [[1,1],[2,1],[-1,1]] # Matrix coefs
rhs = [1, 6, 1] 

for c, s, r, cn in zip(coefs, sense, rhs, const_names):
    expr = lpSum([x[var_names[i]] * c[i] for i in range(2)])
    model += LpConstraint(e=expr, sense = s, name = cn, rhs = r)
    
obj_coefs = [4,2]
model += lpSum([x[var_names[i]] * obj_coefs[i] for i in range(2)])
    
model

some-problem:
MAXIMIZE
4*x_First + 2*x_Second + 0
SUBJECT TO
GE: x_First + x_Second >= 1

LE: 2 x_First + x_Second <= 6

EQ: - x_First + x_Second = 1

VARIABLES
x_First Continuous
x_Second Continuous

In [17]:
status = model.solve()
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
for n in var_names: print(x[n].value())

status: 1, Optimal
objective: 12.000000199999999
1.6666667
2.6666667


# Homework - use the PuLP library to solve the following OR problem 

Johnson & Sons company manufactures 6 types of toys. Each toy is produced by utilizing at least one Machine 1-4, requiring a different production time (see Table below). For instance, product A requires 4 minutes on Machine 1, 4 minutes on Machine 2, 0 Minutes on Machine 3, and 0 minutes on Machine 4 (all machines must be utilized to produce a toy unless the production time equals 0). Each machine is available for a different number of hours per week. The company aims to identify the number (product-mix) of toys that must be manufactured to maximize the income (can be continuous). Notably, each toy can be sold for a different price. Due to market expectations, the company wants to manufacture twice as many F toys as A. Furthermore, the number of toys B should equal C. Solve this problem using the PuLP library. Prepare a report (in the jupyter notebook) containing information on the following:
<ol>
<li>The number of toys to manufacture;</li>
<li>The expected income;</li>
<li>The production time required on each machine;</li>
</ol>
Consider the total and partial values, i.e., considered for each toy A-F separately (e.g., income resulting from selling toy A). Also, answer the following questions concerning the found solution:
<ol>
<li>Which toy(s) production should be focused on?  </li>
<li>Is there a toy that can be excluded from consideration for production? </li>
<li>Is there a machine that is not fully utilized?</li>
</ol>

| Toy | Machine 1 | Machine 2 | Machine 3 | Machine 4 | Selling price |
| --- | --- | --- | --- | --- | --- |
| A | 4 (minutes!) | 4 | 0 | 0 | 2.50 USD |
| B | 0 | 3 | 3 | 0 | 1.00 USD |
| C | 5 | 0 | 2 | 5 | 4.00 USD |
| D | 2 | 4 | 0 | 4 | 3.00 USD |
| E | 0 | 4 | 5 | 2 | 3.50 USD |
| F | 2 | 2 | 1 | 1 | 4.00 USD |
| Production time limit (hours!) | 120 | 60 |  80 |  120 |  |

---------------------------

### *Understanding the problem:*
**Max Z = 2.5A + 1B + 4C + 3D + 3.5E + 4F** <br> <br>
**Constraints:**
1.  4A + 5C + 2D + 2F <= 60 * 120
2.  4A + 3B + 4D + 4E + 2F <= 60 * 60
3.  3B + 2C + 5E + 1F <= 60 * 80
4.  5C + 4D + 2E + 1F <= 60 * 120

5.  B=C -> B-C = 0
6.  F=2A -> F-2A = 0

and nonnegativity constraints: A,B,C,D,E,F >= 0

-------------

In [18]:
model = LpProblem(name = 'Johnson&Sons', sense = LpMaximize)

In [19]:
# The decision variables - in this case they are Toys from A to F
A = LpVariable(name="Toy_A", lowBound=0, cat=LpInteger)
B = LpVariable(name="Toy_B", lowBound=0, cat=LpInteger)
C = LpVariable(name="Toy_C", lowBound=0, cat=LpInteger)
D = LpVariable(name="Toy_D", lowBound=0, cat=LpInteger)
E = LpVariable(name="Toy_E", lowBound=0, cat=LpInteger)
F = LpVariable(name="Toy_F", lowBound=0, cat=LpInteger)


In [20]:
objective_function = 2.5 * A + 1 * B + 4 * C + 3 * D + 3.5 * E + 4 * F
model += objective_function

In [21]:
#constraints:
model += (4 * A + 5 * C + 2 * D + 2 * F <= 60 * 120, "#1 constraint")
model += (4 * A + 3 * B + 4 * D + 4 * E + 2 * F <= 60 * 60, "#2 constraint")
model += (3 * B + 2 * C + 5 * E + 1 * F <= 60 * 80, "#3 constraint")
model += (5 * C + 4 * D + 2 * E + 1 * F <= 60 * 120, "#4 constraint")
model += (B - C == 0, "#5 constraint")
model += (F - 2 * A == 0, "#6 constraint")

In [22]:
model

Johnson&Sons:
MAXIMIZE
2.5*Toy_A + 1*Toy_B + 4*Toy_C + 3*Toy_D + 3.5*Toy_E + 4*Toy_F + 0.0
SUBJECT TO
#1_constraint: 4 Toy_A + 5 Toy_C + 2 Toy_D + 2 Toy_F <= 7200

#2_constraint: 4 Toy_A + 3 Toy_B + 4 Toy_D + 4 Toy_E + 2 Toy_F <= 3600

#3_constraint: 3 Toy_B + 2 Toy_C + 5 Toy_E + Toy_F <= 4800

#4_constraint: 5 Toy_C + 4 Toy_D + 2 Toy_E + Toy_F <= 7200

#5_constraint: Toy_B - Toy_C = 0

#6_constraint: - 2 Toy_A + Toy_F = 0

VARIABLES
0 <= Toy_A Integer
0 <= Toy_B Integer
0 <= Toy_C Integer
0 <= Toy_D Integer
0 <= Toy_E Integer
0 <= Toy_F Integer

### Now we may solve the Johnson&Sons problem

In [23]:
status = model.solve()
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}\n")
for name, constraint in model.constraints.items():  print(f"{name}: {constraint.value()}")

status: 1, Optimal
objective: 5698.0

#1_constraint: -1767.0
#2_constraint: -1.0
#3_constraint: -3.0
#4_constraint: -2403.0
#5_constraint: 0.0
#6_constraint: 0.0


In [24]:
model.variables()

[Toy_A, Toy_B, Toy_C, Toy_D, Toy_E, Toy_F]

In [25]:
for var in model.variables():
    print(f'Value of variable {var}: {var.value()}')

Value of variable Toy_A: 106.0
Value of variable Toy_B: 917.0
Value of variable Toy_C: 917.0
Value of variable Toy_D: 0.0
Value of variable Toy_E: 0.0
Value of variable Toy_F: 212.0


-----------

### Amount of toys

In [26]:
A.value() + B.value() + C.value() + D.value() + E.value() + F.value()

2152.0

### Summary for each toy

In [28]:
#Toy A summary
print(f'Number of toys A produced: {A.value()}')
print(f'Total profit from toys A: {A.value() * 2.5}')

Number of toys A produced: 106.0
Total profit from toys A: 265.0


In [29]:
#Toy B summary
print(f'Number of toys B produced: {B.value()}')
print(f'Total profit from toys B: {B.value() * 1}')

Number of toys B produced: 917.0
Total profit from toys B: 917.0


In [30]:
#Toy C summary
print(f'Number of toys C produced: {C.value()}')
print(f'Total profit from toys C: {C.value() * 4}')

Number of toys C produced: 917.0
Total profit from toys C: 3668.0


In [31]:
#Toy D summary
print(f'Number of toys D produced: {D.value()}')
print(f'Total profit from toys D: {D.value() * 3}')

Number of toys D produced: 0.0
Total profit from toys D: 0.0


In [32]:
#Toy E summary
print(f'Number of toys E produced: {E.value()}')
print(f'Total profit from toys E: {E.value() * 3.5}')

Number of toys E produced: 0.0
Total profit from toys E: 0.0


In [33]:
#Toy F summary
print(f'Number of toys F produced: {F.value()}')
print(f'Total profit from toys F: {F.value() * 4}')

Number of toys F produced: 212.0
Total profit from toys F: 848.0


### Profit

In [34]:
#Total profit summary
print(f'Total profit: {model.objective.value()}')

Total profit: 5698.0


In [35]:
#Maximum profit of a toy
max_profit = max(A.value() * 2.5, B.value() * 1, C.value() * 4, D.value() * 3, E.value() * 3.5, F.value() * 4)
print(f'Maximum profit of one toy type: {max_profit}')

Maximum profit of one toy type: 3668.0


### The production time required on each machine

In [27]:
#first constraint -> Machine 1
print(f'Constraint satisfied: {4 * A.value() + 5 * C.value() + 2 * D.value() + 2 * F.value() <= 60 * 120}')
print(f'Time in minutes that machine 1 used for production: {4 * A.value() + 5 * C.value() + 2 * D.value() + 2 * F.value()}')
print(f'Time that machine was unused: {(60 * 120) - (4 * A.value() + 5 * C.value() + 2 * D.value() + 2 * F.value())}')


Constraint satisfied: True
Time in minutes that machine 1 used for production: 5433.0
Time that machine was unused: 1767.0


In [28]:
#second constraint -> Machine 2
print(f'Constraint satisfied: {4 * A.value() + 3 * B.value() + 4 * D.value() + 4 * E.value() + 2 * F.value() <= 60 * 60}')
print(f'Time in minutes that machine 2 used for production: {4 * A.value() + 3 * B.value() + 4 * D.value() + 4 * E.value() + 2 * F.value()}')
print(f'Time that machine was unused: {(60 * 60) - (4 * A.value() + 3 * B.value() + 4 * D.value() + 4 * E.value() + 2 * F.value())}')

Constraint satisfied: True
Time in minutes that machine 1 used for production: 3599.0
Time that machine was unused: 1.0


In [29]:
#third constraint -> Machine 3
print(f'Constraint satisfied: {3 * B.value() + 2 * C.value() + 5 * E.value() + 1 * F.value() <= 60 * 80}')
print(f'Time in minutes that machine 3 used for production: {3 * B.value() + 2 * C.value() + 5 * E.value() + 1 * F.value()}')
print(f'Time that machine was unused: {(60 * 80) - (3 * B.value() + 2 * C.value() + 5 * E.value() + 1 * F.value())}')

Constraint satisfied: True
Time in minutes that machine 1 used for production: 4797.0
Time that machine was unused: 3.0


In [30]:
#fourth constraint -> Machine 4
print(f'Constraint satisfied: {5 * C.value() + 4 * D.value() + 2 * E.value() + 1 * F.value() <= 60 * 120}')
print(f'Time in minutes that machine 4 used for production: {5 * C.value() + 4 * D.value() + 2 * E.value() + 1 * F.value()}')
print(f'Time that machine was unused: {(60 * 120) - (5 * C.value() + 4 * D.value() + 2 * E.value() + 1 * F.value())}')

Constraint satisfied: True
Time in minutes that machine 1 used for production: 4797.0
Time that machine was unused: 2403.0


In [38]:
F.value()*4

848.0

## Raport 

### 1) Income&Toys
The company Johnson & Sons will produce: 2152 toys,earning **5698.00 USD**; <br> more specifically:

| Toy | Number of toys | Profit per toy | Profit per toy type |
| --- | --- | --- | --- | 
| A | 106 | 2.50 USD | 265.00 USD | 
| B | 917 | 1.00 USD | 917.00 USD |
| C | 917 | 4.00 USD | 3668.00 USD |
| D | 0 | 3.00 USD | 0.00 USD | 
| E | 0 | 3.50 USD | 0.00 USD | 
| F | 212 | 4.00 USD | 848.00 USD | 

### 2) Machines

| Machine | Time Used (minutes)| Time Unused (minutes)| Total (hours) |
| --- | --- | --- | --- | 
| Machine 1 | 5433 | 1767 | 120 |
| Machine 2 | 3599 | 1 | 60 |
| Machine 3 | 4797 | 3 | 80 | 
| Machine 4 | 4797 | 2403 | 120 |


### 3) Questions
1. I will consider 2 ways; <br>
The focus may be put on production of toy F, since it is most profitable, giving 4.00 USD for one toy, as well as toy C; but toy F requires way less resources, i.e. it needs only 6 time units from the machines, wheras toy C requires 12 time units.<br>
Also we have the constraint that the F toy has to be produced in doubled amout of toy A, and they take 6 and 8 time units respectively; on the other hand we have also the constarint that toy C has to be created in the same amout as toy B and they take 12 and 6 time units respectively. <br> 
But after the time constraints on every machine we do actually get that the toy C is the most profitable one.<br>
So the focus should be put on toy C and toy F.
2. Yes, we can exclude from the considerations toys D&E, since based on the calculations actually none of them will be produced in the problem when we want to maximize the Johnson & Sons income.
3. Yes, the machine that was not fully utalized was Machine1 (with Time that machine was unused: 1767.0) and Machine4 (with Time that machine was unused: 2403.0). <br>
Machine2 and Machine3 are almost fully utalized, even though they do have the negative constraints -1 and -3 respectively, one and 3 minutes are not a big difference.<br>
So overall we may lower the production time limits on machines by the values that they are not used, so by the constraints that are negative.  